In [ ]:
from google.colab import files
uploaded = files.upload()

Saving final.csv to final (1).csv


In [ ]:
import pandas as pd

weaning = pd.read_csv('final.csv')

In [ ]:
weaning.head()

,subject_id,hadm_id,stay_id,hadm_no,stay_no,intubation_time,ventilation_starttime,ventilation_endtime,ventilation_time,extubation_time,...,los,mortality,died_on_vent,ventend_death,vent_duration_mins,age,niv_48,reintubation_48,died_48,ext_success
0,10001884,26184834.0,37510196,1,1,2131-01-11 04:31:00,2131-01-11 04:40:00,2131-01-12 17:40:00,2220.0,2131-01-12 17:41:00,...,17796.0,1,0,NaN,2230.0,68,NaN,NaN,NaN,1
1,10001884,26184834.0,37510196,1,1,NaN,2131-01-13 04:00:00,2131-01-19 17:45:00,9465.0,NaN,...,17796.0,1,1,690.0,10155.0,68,NaN,NaN,NaN,0
2,10002428,28662225.0,38875437,1,1,NaN,2156-04-19 20:10:00,2156-04-22 17:05:00,4135.0,2156-04-22 17:11:00,...,24610.0,0,0,NaN,4141.0,80,NaN,NaN,NaN,1
3,10003400,23559586.0,34577403,1,1,NaN,2137-08-10 20:33:00,2137-08-11 13:53:00,1040.0,NaN,...,42778.0,1,1,31872.0,32912.0,72,NaN,NaN,NaN,0
4,10003637,28317408.0,32824762,1,1,NaN,2150-05-20 07:45:00,2150-05-22 18:39:00,3534.0,NaN,...,11314.0,1,1,-134.0,3400.0,57,NaN,NaN,NaN,0


In [ ]:
from google.colab import auth
auth.authenticate_user()

# Import BigQuery client
from google.cloud import bigquery
client = bigquery.Client(project='mimic-ventilation-project-2')

In [ ]:
query = """
SELECT subject_id, stay_id, hadm_id, charttime, itemid, valuenum, value
FROM `physionet-data.mimiciv_icu.chartevents`
WHERE itemid = 223849
"""

# Run the query and load into a DataFrame
chartevents_ventmode = client.query(query).to_dataframe()
print(f"Retrieved {len(chartevents_ventmode)} charted ventilator mode rows.")
chartevents_ventmode.head()

Retrieved 472893 charted ventilator mode rows.


,subject_id,stay_id,hadm_id,charttime,itemid,valuenum,value
0,14918516,31544131,22852565,2177-08-26 15:00:00,223849,26.0,APRV
1,18973018,39142568,24553597,2170-08-24 11:00:00,223849,26.0,APRV
2,16599161,35515008,24881628,2155-12-01 11:00:00,223849,10.0,CPAP
3,19992885,35180845,20352341,2152-07-23 04:00:00,223849,10.0,CPAP
4,17849738,30962143,24496600,2141-09-16 15:00:00,223849,14.0,PCV+


In [ ]:
# rename and convert charttime to datetime
chartevents_ventmode = chartevents_ventmode.rename(columns={
    'value': 'mode',
    'valuenum': 'mode_code'
})
chartevents_ventmode['charttime'] = pd.to_datetime(chartevents_ventmode['charttime'])

# keep only required columns
chartevents_ventmode = chartevents_ventmode[['subject_id', 'hadm_id', 'stay_id', 'charttime', 'mode', 'mode_code']]

# merge with weaning on subject_id and stay_id, keeping all unique columns
weaning_merged = pd.merge(
    weaning,
    chartevents_ventmode,
    on=['subject_id', 'hadm_id', 'stay_id'],
    how='inner',  # change to 'outer' or 'left' if needed
    suffixes=('_ventad', '_chart')
)

# sort by subject_id, hadm_id, stay_id and times chronologically
weaning_merged = weaning_merged.sort_values(
    by=['subject_id', 'hadm_id', 'stay_id', 'intubation_time', 'ventilation_starttime', 'extubation_time', 'charttime']
).reset_index(drop=True)

In [ ]:
weaning_merged = weaning_merged.drop_duplicates()

In [ ]:
weaning_merged.shape

(270994, 25)

In [ ]:
weaning_merged.head(50)

,subject_id,hadm_id,stay_id,hadm_no,stay_no,intubation_time,ventilation_starttime,ventilation_endtime,ventilation_time,extubation_time,...,ventend_death,vent_duration_mins,age,niv_48,reintubation_48,died_48,ext_success,charttime,mode,mode_code
0,10002428,28662225.0,38875437,1,1,NaN,2156-04-19 20:10:00,2156-04-22 17:05:00,4135.0,2156-04-22 17:11:00,...,NaN,4141.0,80,NaN,NaN,NaN,1,2156-04-19 20:00:00,CMV/ASSIST/AutoFlow,49.0
1,10002428,28662225.0,38875437,1,1,NaN,2156-04-19 20:10:00,2156-04-22 17:05:00,4135.0,2156-04-22 17:11:00,...,NaN,4141.0,80,NaN,NaN,NaN,1,2156-04-20 00:00:00,CMV/ASSIST/AutoFlow,49.0
2,10002428,28662225.0,38875437,1,1,NaN,2156-04-19 20:10:00,2156-04-22 17:05:00,4135.0,2156-04-22 17:11:00,...,NaN,4141.0,80,NaN,NaN,NaN,1,2156-04-20 04:00:00,CMV/ASSIST/AutoFlow,49.0
3,10002428,28662225.0,38875437,1,1,NaN,2156-04-19 20:10:00,2156-04-22 17:05:00,4135.0,2156-04-22 17:11:00,...,NaN,4141.0,80,NaN,NaN,NaN,1,2156-04-20 08:00:00,CMV/ASSIST/AutoFlow,49.0
4,10002428,28662225.0,38875437,1,1,NaN,2156-04-19 20:10:00,2156-04-22 17:05:00,4135.0,2156-04-22 17:11:00,...,NaN,4141.0,80,NaN,NaN,NaN,1,2156-04-20 12:00:00,CMV/ASSIST/AutoFlow,49.0
5,10002428,28662225.0,38875437,1,1,NaN,2156-04-19 20:10:00,2156-04-22 17:05:00,4135.0,2156-04-22 17:11:00,...,NaN,4141.0,80,NaN,NaN,NaN,1,2156-04-20 18:00:00,CMV/ASSIST/AutoFlow,49.0
6,10002428,28662225.0,38875437,1,1,NaN,2156-04-19 20:10:00,2156-04-22 17:05:00,4135.0,2156-04-22 17:11:00,...,NaN,4141.0,80,NaN,NaN,NaN,1,2156-04-20 20:00:00,CMV/ASSIST/AutoFlow,49.0
7,10002428,28662225.0,38875437,1,1,NaN,2156-04-19 20:10:00,2156-04-22 17:05:00,4135.0,2156-04-22 17:11:00,...,NaN,4141.0,80,NaN,NaN,NaN,1,2156-04-21 00:00:00,CMV/ASSIST/AutoFlow,49.0
8,10002428,28662225.0,38875437,1,1,NaN,2156-04-19 20:10:00,2156-04-22 17:05:00,4135.0,2156-04-22 17:11:00,...,NaN,4141.0,80,NaN,NaN,NaN,1,2156-04-21 04:00:00,CMV/ASSIST/AutoFlow,49.0
9,10002428,28662225.0,38875437,1,1,NaN,2156-04-19 20:10:00,2156-04-22 17:05:00,4135.0,2156-04-22 17:11:00,...,NaN,4141.0,80,NaN,NaN,NaN,1,2156-04-21 08:00:00,CMV/ASSIST/AutoFlow,49.0


In [ ]:
unique_value_pairs = (
    weaning_merged[['mode', 'mode_code']]
    .dropna(subset=['mode', 'mode_code'])
    .drop_duplicates()
    .sort_values(by=['mode', 'mode_code'])
    .reset_index(drop=True)
)

print("Unique (mode, mode_code) pairs:")
print(unique_value_pairs)

Unique (mode, mode_code) pairs:
                   mode  mode_code
0                  APRV       26.0
1     Apnea Ventilation       17.0
2                   CMV        1.0
3            CMV/ASSIST        2.0
4   CMV/ASSIST/AutoFlow       49.0
5          CMV/AutoFlow       48.0
6                  CPAP       10.0
7              CPAP/PPS       53.0
8              CPAP/PSV       11.0
9                   MMV       12.0
10         MMV/AutoFlow       50.0
11              MMV/PSV       13.0
12     MMV/PSV/AutoFlow       51.0
13                 PCV+       14.0
14             PCV+/PSV       45.0
15           PCV+Assist       71.0
16                 SIMV        6.0
17        SIMV/AutoFlow       46.0
18             SIMV/PSV        7.0
19    SIMV/PSV/AutoFlow       47.0
20      SYNCHRON MASTER       15.0
21       SYNCHRON SLAVE       16.0
22              Standby       30.0


In [ ]:
weaning_merged_df = weaning_merged.copy()

In [ ]:
# define valuenum codes for each group based on categorisation
control_modes = {26.0, 17.0, 1.0, 2.0, 49.0, 48.0, 12.0, 14.0, 45.0, 71.0, 6.0, 46.0, 7.0, 47.0}
support_modes = {10.0, 53.0, 11.0, 3.0}

# function to classify ventilation type
def classify_vent_mode(val):
    if val in control_modes:
        return "Control"
    elif val in support_modes:
        return "Support"
    else:
        return "Other"  # For unclassified modes like MMV, SYNCHRON, etc.

# apply classification
weaning_merged_df['ventilation_type'] = weaning_merged['mode_code'].apply(classify_vent_mode)

# preview the updated DataFrame
weaning_merged_df.head(50)

,subject_id,hadm_id,stay_id,hadm_no,stay_no,intubation_time,ventilation_starttime,ventilation_endtime,ventilation_time,extubation_time,...,vent_duration_mins,age,niv_48,reintubation_48,died_48,ext_success,charttime,mode,mode_code,ventilation_type
0,10002428,28662225.0,38875437,1,1,NaN,2156-04-19 20:10:00,2156-04-22 17:05:00,4135.0,2156-04-22 17:11:00,...,4141.0,80,NaN,NaN,NaN,1,2156-04-19 20:00:00,CMV/ASSIST/AutoFlow,49.0,Control
1,10002428,28662225.0,38875437,1,1,NaN,2156-04-19 20:10:00,2156-04-22 17:05:00,4135.0,2156-04-22 17:11:00,...,4141.0,80,NaN,NaN,NaN,1,2156-04-20 00:00:00,CMV/ASSIST/AutoFlow,49.0,Control
2,10002428,28662225.0,38875437,1,1,NaN,2156-04-19 20:10:00,2156-04-22 17:05:00,4135.0,2156-04-22 17:11:00,...,4141.0,80,NaN,NaN,NaN,1,2156-04-20 04:00:00,CMV/ASSIST/AutoFlow,49.0,Control
3,10002428,28662225.0,38875437,1,1,NaN,2156-04-19 20:10:00,2156-04-22 17:05:00,4135.0,2156-04-22 17:11:00,...,4141.0,80,NaN,NaN,NaN,1,2156-04-20 08:00:00,CMV/ASSIST/AutoFlow,49.0,Control
4,10002428,28662225.0,38875437,1,1,NaN,2156-04-19 20:10:00,2156-04-22 17:05:00,4135.0,2156-04-22 17:11:00,...,4141.0,80,NaN,NaN,NaN,1,2156-04-20 12:00:00,CMV/ASSIST/AutoFlow,49.0,Control
5,10002428,28662225.0,38875437,1,1,NaN,2156-04-19 20:10:00,2156-04-22 17:05:00,4135.0,2156-04-22 17:11:00,...,4141.0,80,NaN,NaN,NaN,1,2156-04-20 18:00:00,CMV/ASSIST/AutoFlow,49.0,Control
6,10002428,28662225.0,38875437,1,1,NaN,2156-04-19 20:10:00,2156-04-22 17:05:00,4135.0,2156-04-22 17:11:00,...,4141.0,80,NaN,NaN,NaN,1,2156-04-20 20:00:00,CMV/ASSIST/AutoFlow,49.0,Control
7,10002428,28662225.0,38875437,1,1,NaN,2156-04-19 20:10:00,2156-04-22 17:05:00,4135.0,2156-04-22 17:11:00,...,4141.0,80,NaN,NaN,NaN,1,2156-04-21 00:00:00,CMV/ASSIST/AutoFlow,49.0,Control
8,10002428,28662225.0,38875437,1,1,NaN,2156-04-19 20:10:00,2156-04-22 17:05:00,4135.0,2156-04-22 17:11:00,...,4141.0,80,NaN,NaN,NaN,1,2156-04-21 04:00:00,CMV/ASSIST/AutoFlow,49.0,Control
9,10002428,28662225.0,38875437,1,1,NaN,2156-04-19 20:10:00,2156-04-22 17:05:00,4135.0,2156-04-22 17:11:00,...,4141.0,80,NaN,NaN,NaN,1,2156-04-21 08:00:00,CMV/ASSIST/AutoFlow,49.0,Control


In [ ]:
# Dividing dataframe into prewean and postwean dataframes on the basis of the first instance of a "support" mode for each subject_id and stay_id combination

# Find index of the first 'Support' for each subject_id and stay_id
first_support_idx = (
    weaning_merged_df[weaning_merged_df['ventilation_type'] == 'Support']
    .groupby(['subject_id', 'stay_id'])
    .head(1)
    .reset_index()[['subject_id', 'stay_id', 'index']]
    .rename(columns={'index': 'support_index'})
)

# Merge back to original dataframe to know the cut-off index for each group
weaning_merged_df = weaning_merged_df.reset_index().rename(columns={'index': 'original_index'})
weaning_with_cutoff = weaning_merged_df.merge(first_support_idx, on=['subject_id', 'stay_id'], how='left')

# Split based on index comparison
prewean = weaning_with_cutoff[
    weaning_with_cutoff['original_index'] < weaning_with_cutoff['support_index']
]

postwean = weaning_with_cutoff[
    weaning_with_cutoff['original_index'] >= weaning_with_cutoff['support_index']
]

In [ ]:
prewean.head(50)

,original_index,subject_id,hadm_id,stay_id,hadm_no,stay_no,intubation_time,ventilation_starttime,ventilation_endtime,ventilation_time,...,age,niv_48,reintubation_48,died_48,ext_success,charttime,mode,mode_code,ventilation_type,support_index
0,0,10002428,28662225.0,38875437,1,1,NaN,2156-04-19 20:10:00,2156-04-22 17:05:00,4135.0,...,80,NaN,NaN,NaN,1,2156-04-19 20:00:00,CMV/ASSIST/AutoFlow,49.0,Control,16.0
1,1,10002428,28662225.0,38875437,1,1,NaN,2156-04-19 20:10:00,2156-04-22 17:05:00,4135.0,...,80,NaN,NaN,NaN,1,2156-04-20 00:00:00,CMV/ASSIST/AutoFlow,49.0,Control,16.0
2,2,10002428,28662225.0,38875437,1,1,NaN,2156-04-19 20:10:00,2156-04-22 17:05:00,4135.0,...,80,NaN,NaN,NaN,1,2156-04-20 04:00:00,CMV/ASSIST/AutoFlow,49.0,Control,16.0
3,3,10002428,28662225.0,38875437,1,1,NaN,2156-04-19 20:10:00,2156-04-22 17:05:00,4135.0,...,80,NaN,NaN,NaN,1,2156-04-20 08:00:00,CMV/ASSIST/AutoFlow,49.0,Control,16.0
4,4,10002428,28662225.0,38875437,1,1,NaN,2156-04-19 20:10:00,2156-04-22 17:05:00,4135.0,...,80,NaN,NaN,NaN,1,2156-04-20 12:00:00,CMV/ASSIST/AutoFlow,49.0,Control,16.0
5,5,10002428,28662225.0,38875437,1,1,NaN,2156-04-19 20:10:00,2156-04-22 17:05:00,4135.0,...,80,NaN,NaN,NaN,1,2156-04-20 18:00:00,CMV/ASSIST/AutoFlow,49.0,Control,16.0
6,6,10002428,28662225.0,38875437,1,1,NaN,2156-04-19 20:10:00,2156-04-22 17:05:00,4135.0,...,80,NaN,NaN,NaN,1,2156-04-20 20:00:00,CMV/ASSIST/AutoFlow,49.0,Control,16.0
7,7,10002428,28662225.0,38875437,1,1,NaN,2156-04-19 20:10:00,2156-04-22 17:05:00,4135.0,...,80,NaN,NaN,NaN,1,2156-04-21 00:00:00,CMV/ASSIST/AutoFlow,49.0,Control,16.0
8,8,10002428,28662225.0,38875437,1,1,NaN,2156-04-19 20:10:00,2156-04-22 17:05:00,4135.0,...,80,NaN,NaN,NaN,1,2156-04-21 04:00:00,CMV/ASSIST/AutoFlow,49.0,Control,16.0
9,9,10002428,28662225.0,38875437,1,1,NaN,2156-04-19 20:10:00,2156-04-22 17:05:00,4135.0,...,80,NaN,NaN,NaN,1,2156-04-21 08:00:00,CMV/ASSIST/AutoFlow,49.0,Control,16.0


In [ ]:
postwean.head(50)

,original_index,subject_id,hadm_id,stay_id,hadm_no,stay_no,intubation_time,ventilation_starttime,ventilation_endtime,ventilation_time,...,age,niv_48,reintubation_48,died_48,ext_success,charttime,mode,mode_code,ventilation_type,support_index
16,16,10002428,28662225.0,38875437,1,1,NaN,2156-04-19 20:10:00,2156-04-22 17:05:00,4135.0,...,80,NaN,NaN,NaN,1,2156-04-22 09:00:00,CPAP/PSV,11.0,Support,16.0
17,17,10002428,28662225.0,38875437,1,1,NaN,2156-04-19 20:10:00,2156-04-22 17:05:00,4135.0,...,80,NaN,NaN,NaN,1,2156-04-22 12:00:00,CPAP/PSV,11.0,Support,16.0
18,18,10002428,28662225.0,38875437,1,1,NaN,2156-04-19 20:10:00,2156-04-22 17:05:00,4135.0,...,80,NaN,NaN,NaN,1,2156-04-22 13:00:00,CPAP/PSV,11.0,Support,16.0
19,19,10002428,28662225.0,38875437,1,1,NaN,2156-04-19 20:10:00,2156-04-22 17:05:00,4135.0,...,80,NaN,NaN,NaN,1,2156-04-22 15:00:00,CPAP/PSV,11.0,Support,16.0
20,20,10002428,28662225.0,38875437,1,1,NaN,2156-04-19 20:10:00,2156-04-22 17:05:00,4135.0,...,80,NaN,NaN,NaN,1,2156-04-22 16:00:00,PSV/SBT,NaN,Other,16.0
25,25,10003400,23559586.0,34577403,1,1,NaN,2137-08-10 20:33:00,2137-08-11 13:53:00,1040.0,...,72,NaN,NaN,NaN,0,2137-08-11 09:00:00,CPAP/PSV,11.0,Support,25.0
26,26,10003400,23559586.0,34577403,1,1,NaN,2137-08-10 20:33:00,2137-08-11 13:53:00,1040.0,...,72,NaN,NaN,NaN,0,2137-08-11 12:00:00,CPAP/PSV,11.0,Support,25.0
31,31,10004235,24181354.0,34100191,1,1,NaN,2196-02-24 16:52:00,2196-02-27 16:28:00,4296.0,...,47,NaN,NaN,NaN,1,2196-02-25 07:00:00,CPAP/PSV,11.0,Support,31.0
32,32,10004235,24181354.0,34100191,1,1,NaN,2196-02-24 16:52:00,2196-02-27 16:28:00,4296.0,...,47,NaN,NaN,NaN,1,2196-02-25 11:00:00,CPAP/PSV,11.0,Support,31.0
33,33,10004235,24181354.0,34100191,1,1,NaN,2196-02-24 16:52:00,2196-02-27 16:28:00,4296.0,...,47,NaN,NaN,NaN,1,2196-02-25 16:00:00,CPAP/PSV,11.0,Support,31.0


In [ ]:
prewean.to_csv('prewean.csv', index=False)
from google.colab import files
files.download('prewean.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
postwean.to_csv('postwean.csv', index=False)
from google.colab import files
files.download('postwean.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>